In [18]:
from langchain_community.chat_models import ChatOllama  # 使用 Ollama 封裝的 LLaMA 模型
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.prompts import PromptTemplate

# 定義輸出結構
class MessageClassification(BaseModel):
    verdict: str = Field(description="Verdict whether the message is Real or Fake")
    confidence: str = Field(description="Confidence level of the judgment (e.g., High, Medium, Low)")
    reason: str = Field(description="Brief explanation of the judgment")

# 使用本地 LLaMA 模型（例如 llama3）
judge_llm = ChatOllama(model="llama3:8b")
logic_llm = ChatOllama(model="phi3")
debater_llm = ChatOllama(model="mistral")

# Json 輸出格式解析器
parser = JsonOutputParser(pydantic_object=MessageClassification)
format_instructions = parser.get_format_instructions()


# 單一 LLM 推理的 Prompt
llm_prompt = PromptTemplate.from_template(
    """
You are a professional fact-checker. Analyze the following message and determine if it is real or fake.

Message:
\"\"\"{message}\"\"\"

Fill out the following JSON strictly without any additional text:

{{
  "verdict": "",        // "Real" or "Fake"
  "confidence": "",     // "High", "Medium", or "Low"
  "reason": ""          // Short explanation (1-2 sentences)
}}

Remember:
- DO NOT add anything outside the JSON.
- DO NOT wrap it in markdown (e.g., ```json).
"""
)

# 讓 judge_llm 匯總所有模型觀點的 Prompt
summary_prompt = PromptTemplate.from_template(
    """
You are the final arbiter. Three experts have evaluated the message. Please summarize their opinions and give your final decision.

Message:
\"\"\"{message}\"\"\"

Expert 1 (Logic-focused model):
{logic_opinion}

Expert 2 (Debate-focused model):
{debate_opinion}

Expert 3 (Your own opinion):
{your_opinion}

Now summarize the opinions, resolve any conflicts, and provide a final classification in this JSON format:
{format_instructions}
"""
)

In [19]:
import pandas as pd
import concurrent.futures
import re
import json

def extract_json(text: str) -> dict:
    # 找出第一組結構為 { ... } 的JSON區塊
    match = re.search(r'{[\s\S]*?}', text)
    if not match:
        raise ValueError("No valid JSON object found in output.")
    json_str = match.group()
    return json.loads(json_str)

def call_llm(llm, prompt):
    response = llm.invoke(prompt)
    return response.content if hasattr(response, "content") else response

# 定義分析函式
def analyze_message_with_multi_llm(message: str):
    logic_input = llm_prompt.format(message=message, format_instructions=format_instructions)
    debate_input = llm_prompt.format(message=message, format_instructions=format_instructions)
    judge_input = llm_prompt.format(message=message, format_instructions=format_instructions)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(call_llm, logic_llm, logic_input): "logic",
            executor.submit(call_llm, debater_llm, debate_input): "debate",
            executor.submit(call_llm, judge_llm, judge_input): "judge"
        }
        results = {}
        for future in concurrent.futures.as_completed(futures):
            key = futures[future]
            results[key] = future.result()

    summary_input = summary_prompt.format(
        message=message,
        logic_opinion=results["logic"],
        debate_opinion=results["debate"],
        your_opinion=results["judge"],
        format_instructions=format_instructions
    )

    final_response = judge_llm.invoke(summary_input)
    result = extract_json(final_response.content)
    return result

# 測試訊息
# 載入資料集
fake_df = pd.read_csv('./raw_data/fake.csv')
first_text = fake_df['text'].iloc[0]
message = f"Breaking: {first_text}"
result = analyze_message_with_multi_llm(message)
print(result)

{'verdict': 'Real', 'confidence': 'High', 'reason': 'The tweet appears to be from an authentic account as per the provided username, Donald J. Trump (@realDonaldTrump), and there is a timestamp that aligns with his known Twitter activity patterns.'}
